In [1]:
print("Ok")

Ok


In [2]:
%pwd

'd:\\Fall24\\LLM\\CodeFiles\\End-to-end-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\Fall24\\LLM\\CodeFiles\\End-to-end-Chatbot'

In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [21]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents


In [22]:
extracted_data=load_pdf_file(data='Data/')

In [9]:
# extracted_data

In [23]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [24]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 2550


In [12]:
# text_chunks

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [26]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [27]:
embeddings = download_hugging_face_embeddings()

C:\Users\hrido\AppData\Local\Temp\ipykernel_12660\1196424635.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [28]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [13]:
# query_result

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [31]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "finalchatbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '18205ea6bb3f463e5768b064074c32f7', 'Date': 'Tue, 03 Dec 2024 03:19:44 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [32]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [33]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [34]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [35]:
docsearch

In [36]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [37]:
retrieved_docs = retriever.invoke("What themes of social hierarchy and family dynamics are presented in The Adventures of Roderick Random?")

In [38]:
retrieved_docs

[Document(id='39cae83b-5758-407f-b7a2-ac6739c81ad4', metadata={'page': 0.0, 'source': 'Data\\THE ADVENTURES OF RODERICK RANDOM.pdf'}, page_content='THE ADVENTURES OF RODERICK RANDOM  \n \n \n \n \nCHAPTER I  \n \n \nOf my Birth and Parentage  \n \n \nI was born in the northern part of this united kingdom, in the house of  \nmy grandfather, a gentleman of considerable fortune and influence, who  \nhad on many occasions signalised himself in behalf of his country; and  \nwas remarkable for his abilities in the law, which he exercised with  \ngreat success in the station of a judge, particularly against beggars,  \nfor whom he had a singular aversion.'),
 Document(id='153accc0-333f-4e53-87b9-a11ab3cdd17b', metadata={'page': 14.0, 'source': 'Data\\THE ADVENTURES OF RODERICK RANDOM.pdf'}, page_content='see, Roderick Random —your own flesh and blood, old gentleman. Don’t lay  \na-stern, you dog,” pulling me forward. My grandfather (who was laid up  \nwith the gout) received this relation, af

In [39]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [40]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [41]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "How does Roderick Random's relationship with his grandfather influence his journey?"})
print(response["answer"])

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [43]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])



I don't know.
